#Comprehend resources notebook
Esse notebok é utilizado para interagir com os recursos do Amazon Comprehend, via boto3.

In [ ]:
import boto3
from sagemaker import get_execution_role
import time
from datetime import datetime

##Clientes e role
A seguinte célula cria os clientes para interação com o Amazon Comprehend e Amazon S3.
O nome da role de execução do SageMaker também é retornada, para sabermos a qual role adicionar as permissões necessárias.

In [ ]:
comprehend = boto3.client('comprehend')
s3 = boto3.client('s3')
role = get_execution_role()
role

##Upload S3
Faz o upload do dataset CSV obtido após a limpeza de dados com o primeiro notebook a um bucket S3.

In [ ]:
file_path = 'reports.csv'
bucket_name = 'comprehend-classification-datasets'
key = 'reports.csv'

response = s3.upload_file(file_path, bucket_name, key)
response

##Classifier do Amazon Comprehend
Abaixo, é feita a request para treinar o classificador do Amazon Comprehend com o dataset armazenado no bucket S3. Esse recurso é o modelo de machine learning que irá aprender com os dados inseridos, se tornando capaz de prever a classificação das reports.

A variável 'comprehend_role_arn' é a role de execução do Amazon Comprehend que deve ser criada para permitir que o serviço acesse o S3.

In [ ]:
comprehend_role_arn = 'arn:aws:iam::id_da_conta:role/nome_da_role'
classifier_response = comprehend.create_document_classifier(
    DocumentClassifierName = 'reports-classifier',
    LanguageCode = 'en',
    InputDataConfig = {
        'S3Uri': 's3://' + bucket_name + '/' + key
    },
    DataAccessRoleArn=comprehend_role_arn
)
classifier_response

Como esse processo pode demorar vários minutos, a seguinte célula realiza um loop com intervalos de 60 segundos para buscar o status do Classifier, e só para quando o treinamento houver terminado ou levado mais de 3 horas.

In [ ]:
classifier_arn = classifier_response['DocumentClassifierArn']
max_time = time.time() + 3*60*60
while time.time() < max_time:
    classifier_description = comprehend.describe_document_classifier(
        DocumentClassifierArn=classifier_arn
    )
    status = classifier_description['DocumentClassifierProperties']['Status']
    current_time = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    print('Status: ' + status, current_time)

    if status != 'TRAINING' and status != 'SUBMITTED':
        break

    time.sleep(60)

##Endpoint do Amazon Comprehend
As seguintes células criam um endpoint para o modelo classificador e seguem a mesma lógica das duas células anteriores, fazendo a request e gerando um loop para acompanhar o status.

O endpoint é o recurso que tornará acessível o modelo treinado. Só é possível criá-lo com o classifier já pronto.

In [ ]:
endpoint_response = comprehend.create_endpoint(
    EndpointName='report-classification-endpoint',
    ModelArn = classifier_arn,
    DesiredInferenceUnits=1
)
endpoint_response

In [ ]:
endpoint_arn = endpoint_response['EndpointArn']

max_time = time.time() + 3*60*60
while time.time() < max_time:
    endpoint_description = comprehend.describe_endpoint(
        EndpointArn=endpoint_arn
    )
    status = endpoint_description['EndpointProperties']['Status']
    current_time = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    print('Status: ' + status, current_time)

    if status != 'CREATING':
        break

    time.sleep(60)